In [ ]:
%load_ext autoreload
%autoreload 2
import os
from numpy import transpose, zeros, array, append
from time import perf_counter
from netCDF4 import Dataset
#from astropy.constants import R_earth
from glob import glob
import shutil
from datetime import datetime,timezone,timedelta
import sys
import re

pathsep=os.path.sep

#import kamodo.readers as readers
from kamodo_ccmc.readers.openggcm_gm_tocdf import openggcm_combine_magnetosphere_files

In [ ]:
cadence=600 
requested_variables=['B_x','B_y','B_z']
new_path=None
#for run in "Yihua_Zheng_090721_1","Yihua_Zheng_090721_2":
#for run in ["Zvi_Kantorovich_022121_1"]:
#for run in sys.argv[1:]:
for run in ["Yihua_Zheng_040122_1"]:
#for run in ["Yihua_Zheng_021022_1","Yihua_Zheng_021022_2"]:
    # step 1 get the run directory on the CCMC archive
    run_path="/Users/lrastaet/Kamodo_data/OpenGGCM_GM/"+run
    # grep from Dataset_locations.txt then plit off second column
    datasetlocations_file="/ccmcshare/Web/Public_cgi-bin/Dataset_locations.txt"
    if os.path.exists(datasetlocations_file):
        with open(datasetlocations_file) as f:
            for line in f.readlines():
                if re.search(run, line):
                    run_path=(line.split("\t"))[1]
                    break
    if not run_path:
        break
    # step 2 determine the start and end dates requested to generate list of hourly increments
    date_times=[]
    start_date=None
    end_date=None

    # global magnetosphere runs: we use date and hour
    with open(run_path+pathsep+"DatabaseInfo") as f:
        for line in f.readlines():
            if re.search("# start_time", line):
                start_date=(line.split("\t"))[0]

            if re.search("# end_time", line):
                end_date=(line.split("\t"))[0]

    if not start_date and not end_date:
        break
    start_datetime=datetime.strptime(start_date[0:13],'%Y/%m/%d %H').replace(tzinfo=timezone.utc)
    end_datetime=datetime.strptime(end_date[0:13], '%Y/%m/%d %H').replace(tzinfo=timezone.utc)
    time_interval_hours=int((end_datetime-start_datetime).total_seconds()/3600)
    
    for this_hour in range(time_interval_hours):
        hours_added = timedelta(hours = this_hour)
        date_times.append( (start_datetime+hours_added).strftime("%Y-%m-%d_%H") )
                
#for run in ["Yihua_Zheng_090721_1"]:
    #for date_time in "2015-10-16_11-0", "2015-10-16_11-1", "2015-10-16_11-2":
    #for date_time in ["2015-10-16_11"]:
    #for date_time in ["2015-10-16_12"]:
#    for date_time in ["2020-08-29_00","2020-08-29_01","2020-08-29_02"]:
#    for date_time in ["2018-04-29_04","2018-04-29_05"]: 
    for date_time in date_times:
        #full_file_prefix="/raidz/RoR_data/GM/OpenGGCM/2021/"+run+"/3df/"+run+".3df_"+date_time
        full_file_prefix=run_path+"/3df/"+run+".3df_"+date_time
        file_path=os.path.dirname(full_file_prefix)
        file_prefix=os.path.basename(full_file_prefix)
        
        conversion_succeeded = openggcm_combine_magnetosphere_files(full_file_prefix,cadence=cadence,requested_variables=requested_variables,verbose=True)
        #conversion_succeeded = openggcm_combine_magnetosphere_files(full_file_prefix,verbose=True)

        if conversion_succeeded and new_path is not None:
            nc_file=file_prefix+'.nc'
            old_nc_file=file_path+pathsep+nc_file
            new_nc_file=new_path+pathsep+run+pathsep+nc_file
            if not os.path.isdir(new_path+pathsep+run):
                os.mkdir(new_path+pathsep+run)
            shutil.copy(old_nc_file,new_nc_file)



In [ ]:
from netCDF4 import Dataset
data=Dataset("")